In [19]:
%config Completer.use_jedi = False

In [20]:
import random
import time
import math
import copy
import torch
import torch.nn as nn
import torch.optim as optim
from torch.nn.utils.rnn import pad_sequence
from torchtext.data.metrics import bleu_score
from tqdm import tqdm

In [21]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [22]:
hyp_params = {
    "batch_size": 128,
    "num_epochs": 100,

    # Encoder parameters
    "encoder_embedding_size": 256,
    "encoder_dropout": 0.5,

    # Decoder parameters
    "decoder_dropout": 0.5,
    "decoder_embedding_size": 256,

    # Common parameters
    "hidden_size": 512,
    "num_layers": 2
}

In [23]:
class Encoder(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_size, num_layers, dropout):
        super(Encoder, self).__init__()
        
        self.dropout = nn.Dropout(dropout)
        
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        
        self.LSTM = nn.LSTM(embedding_dim, hidden_size, num_layers, dropout=dropout)
        
    def forward(self, x):
        embedding = self.dropout(self.embedding(x))
        
        outputs, (hidden_state, cell_state) = self.LSTM(embedding)

        return hidden_state, cell_state
    
class Decoder(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_size, num_layers, dropout, output_size):
        super(Decoder, self).__init__()
        
        self.dropout = nn.Dropout(dropout)
        
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        
        self.LSTM = nn.LSTM(embedding_dim, hidden_size, num_layers, dropout=dropout)
        
        self.fc = nn.Linear(hidden_size, output_size)
    
    def forward(self, x, hidden_state, cell_state):
        # as we are not feeding whole sentence we will each token a time
        # hence our sequence length would be just 1 however shape of x is batch_size
        # to add sequence length we will unsequeeze it
        x = x.unsqueeze(0)
        
        embedded = self.dropout(self.embedding(x))
        
        outputs, (hidden_state, cell_state) = self.LSTM(embedded, (hidden_state, cell_state))

        # Shape --> predictions  (1, batch_size, target_vocab_size)
        predictions = self.fc(outputs.squeeze(0))

        return predictions, hidden_state, cell_state

class SeqtoSeq(nn.Module):
    def __init__(self, gen_params, target_vocab, device):
        super(SeqtoSeq, self).__init__()

        self.Encoder = Encoder(gen_params["input_size_encoder"],
                          gen_params["encoder_embedding_size"],
                          gen_params["hidden_size"],
                          gen_params["num_layers"],
                          gen_params["encoder_dropout"]).to(device)

        self.Decoder = Decoder(gen_params["input_size_decoder"],
                          gen_params["decoder_embedding_size"],
                          gen_params["hidden_size"],
                          gen_params["num_layers"],
                          gen_params["decoder_dropout"],
                          gen_params["output_size"]).to(device)

        self.target_vocab = target_vocab
        self.device = device
    
    def forward(self, source, target, tfr=0.5):
        # Shape -> (Sentence length, Batch_size)
        batch_size = source.shape[1]

        target_len = target.shape[0]  # Length of target sentences
        target_vocab_size = len(self.target_vocab)
        
        # here we will store all the outputs
        # so outputs is arrange in a way that sentences are in column and batch size is row and every element
        # will consist of probability of each word from the vocab
        outputs = torch.zeros(target_len, batch_size, target_vocab_size).to(self.device)

        # Shape --> (hs, cs) (num_layers, batch_size size, hidden_size) (contains encoder's hs, cs - context vectors)
        hidden_state, cell_state = self.Encoder(source)

        # Shape of x (32 elements)
        x = target[0]  # First token (Trigger)

        for i in range(1, target_len):
            # Shape --> output (batch_size, target_vocab_size)
            output, hidden_state, cell_state = self.Decoder(x, hidden_state, cell_state)
            outputs[i] = output
            best_guess = output.argmax(1)  # 0th dimension is batch size, 1st dimension is word embedding
            # Schedule sampling
            x = target[
                i] if random.random() < tfr else best_guess  # Either pass the next word correctly from the dataset
            # or use the earlier predicted word

        # Shape --> (sentence length, batch size, vocab size)
        return outputs

In [24]:
from torch.utils.data import Dataset
from collections import Counter
from torchtext.vocab import build_vocab_from_iterator
from torchtext.data.utils import get_tokenizer

class nmtDataset(Dataset):
    def __init__(self, ds_path, split, train_ds=None):
        self.en = open(ds_path + split + '.en', encoding='utf-8').readlines()
        self.de = open(ds_path + split + '.de', encoding='utf-8').readlines()
        
        self.tokenizers = { 'en': get_tokenizer('spacy', language='en_core_web_sm'), 
                            'de': get_tokenizer('spacy', language='de_core_news_sm') }
        
        if split == 'train':
            self.src_vocab, self.trg_vocab = self._build_vocab()
        else:
            self.src_vocab, self.trg_vocab = train_ds.src_vocab, train_ds.trg_vocab
    
    def __len__(self):
        return len(self.en)

    def __getitem__(self, item):
        src_tokens = self.tokenizers['en'](self.en[item].lower().strip())
        trg_tokens = self.tokenizers['de'](self.de[item].lower().strip())
  
        return {
            "src": [self.src_vocab['<sos>']] + self.src_vocab.lookup_indices(src_tokens) + [self.src_vocab['<eos>']],
            "trg": [self.trg_vocab['<sos>']] + self.trg_vocab.lookup_indices(trg_tokens) + [self.trg_vocab['<eos>']]
        }
        
    def _build_vocab(self):
        src_vocab = build_vocab_from_iterator(self._get_tokens(self.en, 'en'), specials=['<unk>', '<pad>', '<sos>', '<eos>'], min_freq=2)  # discarding words occurs < 2 times
        trg_vocab = build_vocab_from_iterator(self._get_tokens(self.de, 'de'), specials=['<unk>', '<pad>', '<sos>', '<eos>'], min_freq=2)
  
        trg_vocab.set_default_index(trg_vocab['<unk>'])
        src_vocab.set_default_index(src_vocab['<unk>'])

        return src_vocab, trg_vocab
    
    def _get_tokens(self, corpus, lang):
        for line in corpus:
            yield self.tokenizers[lang](line.lower().strip())
    
nmtds_train = nmtDataset('datasets/Multi30k/', 'train')
nmtds_valid = nmtDataset('datasets/Multi30k/', 'val', nmtds_train)
nmtds_test = nmtDataset('datasets/Multi30k/', 'test', nmtds_train)

In [25]:
from torch.utils.data import DataLoader


def collate_fn(batch, device):
    PAD_IDX = 1
    trgs = []
    srcs = []
    for row in batch:
        srcs.append(torch.tensor(row["src"]).to(device))
        trgs.append(torch.tensor(row["trg"]).to(device))

    padded_srcs = pad_sequence(srcs, padding_value=PAD_IDX)
    padded_trgs = pad_sequence(trgs, padding_value=PAD_IDX)
    return {"src": padded_srcs, "trg": padded_trgs}

train_dataloader = DataLoader(nmtds_train, batch_size=hyp_params['batch_size'], shuffle=True,
                              collate_fn=lambda batch_size: collate_fn(batch_size, device))

valid_dataloader = DataLoader(nmtds_valid, batch_size=hyp_params['batch_size'], shuffle=True,
                              collate_fn=lambda batch_size: collate_fn(batch_size, device))

In [26]:
def evaluate_generator(model, data_loader, criterion):
    model.eval()
    epoch_loss = 0
    with torch.no_grad():
        for batch_idx, batch in enumerate(data_loader):
            src = batch["src"]  # shape --> e.g. (19, 2) sentence len, batch size
            trg = batch["trg"]  # shape --> e.g. (3, 2) sentence len, batch size

            # Pass the input and target for model's forward method
            # Shape --> (sentence len of TRG, batch size, vocab size) e.g (3, 2, 196)
            # Explanation:
            #    It just outputs probabilities for every single word in our vocab
            #    for each word in sentence and each sentence in batch size
            output = model(src, trg, 0)

            # Updating output shape --> [sentence length * batch size , vocab size]
            # e.g (6, 196)
            output = output.reshape(-1, output.shape[2])

            # sentence len  * batch size
            target = trg.reshape(-1)

            # Calculate the loss value for every epoch
            loss = criterion(output, target)

            epoch_loss += loss.item()

    return epoch_loss/len(data_loader)

In [27]:
hyp_params["input_size_encoder"] = len(nmtds_train.src_vocab)
hyp_params["input_size_decoder"] = len(nmtds_train.trg_vocab)
hyp_params["output_size"] = len(nmtds_train.trg_vocab)

model = SeqtoSeq(hyp_params, target_vocab=nmtds_train.trg_vocab, device=device)
optimizer = optim.Adam(model.parameters())

pad_idx = nmtds_train.trg_vocab["<pad>"]
criterion = nn.CrossEntropyLoss(ignore_index=pad_idx).to(device)

In [ ]:
min_el = math.inf
patience = 1
best_model = {}
best_epoch = 0

epoch_loss = 0
for epoch in range(hyp_params["num_epochs"]):
    model.train()
    for batch_idx, batch in enumerate(tqdm(train_dataloader)):
        src = batch["src"]  # shape --> e.g. (19, 2) sentence len, batch size
        trg = batch["trg"]  # shape --> e.g. (3, 2) sentence len, batch size

        # Clear the accumulating gradients
        optimizer.zero_grad()

        # Pass the input and target for model's forward method
        # Shape --> (sentence len of TRG, batch size, vocab size) e.g (3, 2, 196)
        # Explanation:
        #    It just outputs probabilities for every single word in our vocab
        #    for each word in sentence and each sentence in batch size
        output = model(src, trg)

        # Updating output shape --> [sentence length * batch size , vocab size]
        # e.g (6, 196)
        output = output.reshape(-1, output.shape[2])

        # sentence len  * batch size
        target = trg.reshape(-1)

        # Calculate the loss value for every epoch
        loss = criterion(output, target)

        # Calculate the gradients for weights & biases using back-propagation
        loss.backward()

        # Clip the gradient value is it exceeds > 1
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1)

        # Update the weights values
        optimizer.step()
        
        epoch_loss += loss.item()
    
    epoch_loss /= len(train_dataloader)
    eval_loss = evaluate_generator(model, valid_dataloader, criterion)
    
    print(f"Epoch: {epoch+1}, Train loss: {epoch_loss}, Eval loss: {eval_loss}, patience: {patience}")
    
    if eval_loss < min_el:
        best_epoch = epoch
        min_el = eval_loss
        best_model = copy.deepcopy(model)
        torch.save({
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'eval_loss': min_el
        }, 'model-vanilla.pt')
        patience = 1
    else:
        patience += 1
    
    if patience == 10:
        print("[STOPPING] Early stopping in action..")
        print(f"Best epoch was {best_epoch} with {min_el} eval loss")
        break


In [29]:
def translate(snt, dataset, model):
    tokens = dataset.tokenizers['en'](snt.lower().strip())
    indices = [dataset.src_vocab['<sos>']] + dataset.src_vocab.lookup_indices(tokens) + [dataset.src_vocab['<eos>']]
    inp_tensor = torch.tensor(indices).unsqueeze(1).to(device)

    # Build encoder hidden, cell state
    with torch.no_grad():
        hidden, cell = model.Encoder(inp_tensor)

    outputs = [dataset.trg_vocab["<sos>"]]

    for _ in range(50):
        previous_word = torch.LongTensor([outputs[-1]]).to(device)

        with torch.no_grad():
            output, hidden, cell = model.Decoder(previous_word, hidden, cell)
            best_guess = output.argmax(1).item()

        outputs.append(best_guess)

        # Model predicts it's the end of the sentence
        if output.argmax(1).item() == dataset.trg_vocab['<eos>']:
            break

    return dataset.trg_vocab.lookup_tokens(outputs)

def bleu(model, dataset, device):
    targets = []
    outputs = []

    for example in tqdm(dataset):
        src = example["src"][1:-1]
        trg = example["trg"][1:-1]
        
        src = ' '.join(dataset.src_vocab.lookup_tokens(src))
        trg = dataset.trg_vocab.lookup_tokens(trg)

        prediction = translate(src, dataset, model)
        prediction = prediction[1:-1]  # remove <eos> token
        
        targets.append([trg])
        outputs.append(prediction)

    return bleu_score(outputs, targets)

In [32]:
model_l = SeqtoSeq(hyp_params, target_vocab=nmtds_train.trg_vocab, device=device)
model_l.load_state_dict(torch.load('model-vanilla.pt', map_location=device)["model_state_dict"])

<All keys matched successfully>

In [33]:
bleu(model_l, nmtds_test, device)

100%|██████████| 1000/1000 [00:43<00:00, 23.06it/s]


0.16437269271063837

In [34]:
torch.load('model-vanilla.pt', map_location=device)["eval_loss"]

3.964359372854233